In [3]:

import requests
import datetime
import webbrowser
from pprint import pprint
import os
import sys
import pandas as pd
import numpy as np
from numpy import arctan

import holidays
import csv
import json

def makeLink(station_ID,date):
    year,month,day = (str(date.year), str(date.month),str(date.day))
    link = ("https://www.wunderground.com/weatherstation/WXDailyHistory.asp?ID="+station_ID+
                "&day="+str(day)+
                "&month="+str(month)+
                "&year="+str(year)+
                "&graphspan=day&format=1")
    return link
def calcWB(dry_bulb, humidity):
    return round(dry_bulb*arctan(0.151977*((humidity)+8.313659)**(0.5))+arctan(dry_bulb+humidity)-arctan((humidity)-1.676331)+((0.00391838*(humidity)**(1.5))*arctan(0.023101*humidity))-4.686035,2)
def f(x):
    return (calcWB(float(x['TemperatureF']),float(x['Humidity'])))

def date_generator(begin_datetime, end_datetime):
    '''a generator to yield the dates between the start
    and finish dates by an interval of one day in the
    datetime format.'''
    while begin_datetime <= end_datetime:
        yield begin_datetime
        begin_datetime += datetime.timedelta(days=1)


#rx = getfromLink("https://www.wunderground.com/weatherstation/WXDailyHistory.asp?ID=KNYNEWYO651&day=11&month=11&year=2017&graphspan=day&format=1")
station_id = 'KTXAUSTI646'
begin_datetime = datetime.datetime.strptime("2016/1/1",'%Y/%m/%d').date()
end_datetime = datetime.datetime.strptime("2018/1/1",'%Y/%m/%d').date()
input_filename = 'MSKCCwebids.csv'
fieldnames = ("name","url","api_key","output_webid","forecast_webid")


def get_df(lnk):
    df = pd.read_csv(lnk, ',', index_col=False)
    df = df.dropna(thresh=5)
    return df

def build_df(source_df, new_df):
    '''join dfs'''
    source_df = source_df.append(new_df)
    return source_df

### Main ###

link=makeLink(station_id,begin_datetime)
main_df = get_df(link)

for date in date_generator(begin_datetime, end_datetime):
    lnk = makeLink(station_id,date)
    new_df = get_df(lnk)
    main_df = build_df(main_df, new_df)
#main_df.head()


# Filtering

fil_df = main_df[['Time', 'TemperatureF', 'Humidity']].copy()
fil_df['DryBulb'] = [val for val in fil_df['TemperatureF']]
fil_df['WetBulb'] = fil_df.apply(f, axis=1)
fil_df['DateTimex'] = fil_df['Time']
# Date filtering
fil_df['DateTimex'] = fil_df['DateTimex'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')) # Converts dates from str to datetime
fil_df['DateTimex'] = fil_df['DateTimex'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour, 0, 0)+ datetime.timedelta(hours=int(round(float(dt.minute)/60)))) # Rounds all dates to nearest hour
fil_df = fil_df.drop_duplicates('DateTimex').reset_index() # Drop all rows (except one) for rows with matching dates, then reindex)

fil_df['Hour'] = pd.Series([date.hour for date in fil_df['DateTimex']])
fil_df['Minute'] = pd.Series([date.hour*60 for date in fil_df['DateTimex']])
fil_df['Day'] = pd.Series([(date+datetime.timedelta(days=1)).isoweekday() for date in fil_df['DateTimex']])
fil_df['Weekend'] = pd.Series([1 if date == 1 or date == 7 else 0 for date in fil_df['Day']])
fil_df.rename(columns={'DateTimex': ''}, inplace=True)

In [6]:
fil_df
fil_df.to_csv("Weatherdata2016_2018.csv", sep='\t', encoding='utf-8')